In [1]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve

In [2]:
data_reader=ReadData('/root/autodl-tmp/hallscope/pope/popular/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,label,question_id,category,most_likely,responses
0,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a person in the image?\nAnswer the q...,"[tensor(1), tensor(0), tensor(1), tensor(0), t...","[tensor(989), tensor(990), tensor(991), tensor...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...,None
1,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a dining table in the image?\nAnswer...,"[tensor(1), tensor(0), tensor(1), tensor(0), t...","[tensor(1373), tensor(1374), tensor(1375), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...,None
2,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a person in the image?\nAnswer the q...,"[tensor(1), tensor(0), tensor(1), tensor(0), t...","[tensor(6619), tensor(6620), tensor(6621), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...,None
3,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a chair in the image?\nAnswer the qu...,"[tensor(1), tensor(0), tensor(1), tensor(0), t...","[tensor(1303), tensor(1304), tensor(1305), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...,None
4,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a refrigerator in the image?\nAnswer...,"[tensor(1), tensor(0), tensor(1), tensor(0), t...","[tensor(3787), tensor(3788), tensor(3789), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...,None


In [5]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [6]:
embeddings=np.concatenate(embeddings,axis=0)

In [7]:
embeddings.shape

(13624, 33, 4096)

In [8]:
gt_label=df['label'].to_list()
gt_label=np.concatenate(gt_label,axis=0)

In [9]:
gt_label.shape

(13624,)

In [10]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
resps=[]
for x in responses:
    resps+=x

In [11]:
is_correct=[]
for i,x in enumerate(resps):
    if x.lower()=='no' and gt_label[i]==0:
        is_correct.append(1)
    elif x.lower()=='yes' and gt_label[i]==1:
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [12]:
is_correct.shape

(13624,)

In [13]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 13624 responses. 11850 answer correctly and 1774 answer wrongly


In [14]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 1000
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:9899
validset length:1000
testset length:2725
trainset postive: 8613 negative: 1286
valset postive: 869 negative: 131
testset postive: 2368 negative: 357


In [15]:
all_embeddings = embeddings
all_embeddings.shape

(13624, 33, 4096)

In [16]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [17]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

Grid Search for Best 'n_components' & 'layer'

In [18]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(n_components,X[:,i_layer,:])
    scores=discriminator.get_score()
    return auroc(scores,y)

In [ ]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,12),
    'layer':range(embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

/root/miniconda3/envs/dl3.9/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:794: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


found best n_components:1,layer:0 Score: 0.5
found best n_components:1,layer:2 Score: 0.5112219889493056
found best n_components:1,layer:3 Score: 0.5509623239838719
found best n_components:1,layer:8 Score: 0.6283347534676166
found best n_components:1,layer:15 Score: 0.8066831226556804
found best n_components:1,layer:16 Score: 0.8676464129164873
found best n_components:1,layer:17 Score: 0.872846739693778


...

/root/miniconda3/envs/dl3.9/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:794: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


found best n_components:2,layer:16 Score: 0.8739272129937895
found best n_components:2,layer:17 Score: 0.8807087202101213


  0%|          | 0/33 [00:00<?, ?it/s]

/root/miniconda3/envs/dl3.9/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:794: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


found best n_components:3,layer:17 Score: 0.8808317009109358


In [21]:
best_paras

{'n_components': 1, 'layer': 17, 'best_score': np.float64(0.8505257424959812)}

In [22]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [23]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(hall_label_val)

In [24]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_train[:,best_layer,:])
scores=discriminator.get_score()
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.8650368724113546 AUROC:0.8585901918720695


In [25]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.845137614678899 AUROC:0.8566851646318905


Train Linear Prob to Get Better Results

In [26]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [27]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [28]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} AUROC:{roc_auc_score(y_test,scores_test)}')

test set acc: 0.8755963302752293 AUROC:0.7877685287951659
